## Preprocessing

In [ ]:
!pip install keras_tuner

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import pandas as pd

Using TensorFlow backend


## Method One: Hyperparameter Tuning
The first method used to optimize the neural network model is through hyperparameter tuning. The create_model function below allows kerastuner to decide which activation function to use in the hidden layers, the number of neurons in the first layer, and the number of hidden layers and neurons in hidden layers.

Here we use the same input data as AlphabetSoupCharity.ipynb uses.

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
apps_cutoff_val = 528
application_types_to_replace = []

num_apps_arr = application_df['APPLICATION_TYPE'].value_counts()
index_apps_arr = num_apps_arr.index

for i in range(len(application_df['APPLICATION_TYPE'].value_counts())):
  if num_apps_arr[i] < apps_cutoff_val:
    application_types_to_replace.append(index_apps_arr[i])

application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [4]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff_val = 1000
classifications_to_replace = []

num_class_arr = application_df['CLASSIFICATION'].value_counts()
index_class_arr = num_class_arr.index

for i in range(len(application_df['CLASSIFICATION'].value_counts())):
  if num_class_arr[i] < classification_cutoff_val:
    classifications_to_replace.append(index_class_arr[i])

classifications_to_replace

['C7000',
 'C1700',
 'C4000',
 'C5000',
 'C1270',
 'C2700',
 'C2800',
 'C7100',
 'C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [6]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
prefixes = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

dummies = pd.get_dummies(application_df, columns=prefixes, prefix=prefixes)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [8]:
# Split our preprocessed data into our features and target arrays
features = dummies.drop(columns='IS_SUCCESSFUL', axis=1)
target = dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, stratify=target)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [11]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [12]:
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 10s]
val_accuracy: 0.7254810333251953

Best val_accuracy So Far: 0.7276967763900757
Total elapsed time: 00h 24m 26s


In [13]:
# Get top model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(1)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 26, 'num_layers': 1, 'units_0': 6, 'units_1': 1, 'units_2': 16, 'units_3': 26, 'units_4': 16, 'tuner/epochs': 7, 'tuner/initial_epoch': 3, 'tuner/bracket': 2, 'tuner/round': 1, 'tuner/trial_id': '0031'}


In [14]:
# Evaluate the best model against the test dataset
top_model = tuner.get_best_models(1)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5558 - accuracy: 0.7277 - 584ms/epoch - 2ms/step
Loss: 0.5558351278305054, Accuracy: 0.7276967763900757


In [16]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 26,
 'num_layers': 1,
 'units_0': 6,
 'units_1': 1,
 'units_2': 16,
 'units_3': 26,
 'units_4': 16,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0031'}

## Method Two: Changing Input Data & Reducing the Cutoff Value
1. Dropping CLASSIFICATION and SPECIAL_CONSIDERATIONS columns in addition to EIN and NAME.
2. Cutoff value for apps is 10.
3. Activation function: relu


In [66]:
# Dropping additional columns CLASSIFICATION
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns = ['CLASSIFICATION', 'SPECIAL_CONSIDERATIONS', 'NAME', 'EIN'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,142590,1


In [67]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
apps_cutoff_val = 10
application_types_to_replace = []

num_apps_arr = application_df['APPLICATION_TYPE'].value_counts()
index_apps_arr = num_apps_arr.index

for i in range(len(application_df['APPLICATION_TYPE'].value_counts())):
  if num_apps_arr[i] < apps_cutoff_val:
    application_types_to_replace.append(index_apps_arr[i])

application_types_to_replace

['T25', 'T14', 'T29', 'T15', 'T17']

In [68]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [69]:
# Review the dataframe
application_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,ProductDev,Association,1,0,5000,1
1,T3,Independent,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,Heathcare,Trust,1,100000-499999,142590,1


In [70]:
# Convert categorical data to numeric with `pd.get_dummies`
prefixes = ['APPLICATION_TYPE', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT']

dummies = pd.get_dummies(application_df, columns=prefixes, prefix=prefixes)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0


In [71]:
# Split our preprocessed data into our features and target arrays
features = dummies.drop(columns='IS_SUCCESSFUL', axis=1)
target = dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, stratify=target)

In [72]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
num_cols = dummies.shape[1]
num_cols

40

In [74]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=39))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 80)                3200      
                                                                 
 dense_18 (Dense)            (None, 30)                2430      
                                                                 
 dense_19 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5661 (22.11 KB)
Trainable params: 5661 (22.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5789 - accuracy: 0.7209
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5694 - accuracy: 0.7255
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5677 - accuracy: 0.7280
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5669 - accuracy: 0.7287
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5655 - accuracy: 0.7289
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5649 - accuracy: 0.7289
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5653 - accuracy: 0.7280
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5647 - accuracy: 0.7295
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5645 - accuracy: 0.7299
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5646 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5578 - accuracy: 0.7327
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5580 - accuracy: 0.7330
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5579 - accuracy: 0.7329
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5580 - accuracy: 0.7332
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5576 - accuracy: 0.7332
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7324
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7335
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7324
Epoch 89/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5583 - accuracy: 0.7330
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5578 - accuracy: 

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5782 - accuracy: 0.7181 - 455ms/epoch - 2ms/step
Loss: 0.5781872868537903, Accuracy: 0.7181341052055359


In [78]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization1.h5")

C:\Users\daian\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Method Three: Reducing the Cutoff Value for Applications and Classification.
1. Dropping EIN and NAME columns.
2. Cutoff value for apps is 10.
3. Cutoff value for class is 70.
4. Activation function: tanh

In [50]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [51]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [52]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
apps_cutoff_val = 10
application_types_to_replace = []

num_apps_arr = application_df['APPLICATION_TYPE'].value_counts()
index_apps_arr = num_apps_arr.index

for i in range(len(application_df['APPLICATION_TYPE'].value_counts())):
  if num_apps_arr[i] < apps_cutoff_val:
    application_types_to_replace.append(index_apps_arr[i])

application_types_to_replace

['T25', 'T14', 'T29', 'T15', 'T17']

In [53]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
T13         66
T12         27
T2          16
Other       11
Name: APPLICATION_TYPE, dtype: int64

In [54]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff_val = 70
classifications_to_replace = []

num_class_arr = application_df['CLASSIFICATION'].value_counts()
index_class_arr = num_class_arr.index

for i in range(len(application_df['CLASSIFICATION'].value_counts())):
  if num_class_arr[i] < classification_cutoff_val:
    classifications_to_replace.append(index_class_arr[i])

classifications_to_replace

['C1300',
 'C1280',
 'C1230',
 'C1400',
 'C7200',
 'C2300',
 'C1240',
 'C8000',
 'C7120',
 'C1500',
 'C1800',
 'C6000',
 'C1250',
 'C8200',
 'C1238',
 'C1278',
 'C1235',
 'C1237',
 'C7210',
 'C2400',
 'C1720',
 'C4100',
 'C1257',
 'C1600',
 'C1260',
 'C2710',
 'C0',
 'C3200',
 'C1234',
 'C1246',
 'C1267',
 'C1256',
 'C2190',
 'C4200',
 'C2600',
 'C5200',
 'C1370',
 'C1248',
 'C6100',
 'C1820',
 'C1900',
 'C1236',
 'C3700',
 'C2570',
 'C1580',
 'C1245',
 'C2500',
 'C1570',
 'C1283',
 'C2380',
 'C1732',
 'C1728',
 'C2170',
 'C4120',
 'C8210',
 'C2561',
 'C4500',
 'C2150']

In [55]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      499
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
Name: CLASSIFICATION, dtype: int64

In [56]:
# Convert categorical data to numeric with `pd.get_dummies`
prefixes = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']

dummies = pd.get_dummies(application_df, columns=prefixes, prefix=prefixes)
dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T3,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,0


In [57]:
# Split our preprocessed data into our features and target arrays
features = dummies.drop(columns='IS_SUCCESSFUL', axis=1)
target = dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42, stratify=target)

In [58]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [59]:
X_train_scaled.shape

(25724, 55)

In [60]:
# Find the number of columns in the features data
num_cols = features.shape[1]
num_cols

55

In [61]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="tanh", input_dim=num_cols))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="tanh"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 80)                4480      
                                                                 
 dense_14 (Dense)            (None, 50)                4050      
                                                                 
 dense_15 (Dense)            (None, 30)                1530      
                                                                 
 dense_16 (Dense)            (None, 1)                 31        
                                                                 
Total params: 10091 (39.42 KB)
Trainable params: 10091 (39.42 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [63]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5666 - accuracy: 0.7239
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5529 - accuracy: 0.7310
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7326
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5460 - accuracy: 0.7335
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5441 - accuracy: 0.7339
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5427 - accuracy: 0.7352
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accuracy: 0.7357
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5408 - accuracy: 0.7371
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5404 - accuracy: 0.7371
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5388 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5248 - accuracy: 0.7450
Epoch 82/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5246 - accuracy: 0.7451
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5250 - accuracy: 0.7455
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5248 - accuracy: 0.7455
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5250 - accuracy: 0.7456
Epoch 86/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5248 - accuracy: 0.7454
Epoch 87/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5246 - accuracy: 0.7458
Epoch 88/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5248 - accuracy: 0.7453
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5246 - accuracy: 0.7456
Epoch 90/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5247 - accuracy: 

In [64]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5617 - accuracy: 0.7237 - 514ms/epoch - 2ms/step
Loss: 0.5616834759712219, Accuracy: 0.7237317562103271


In [65]:
nn.save("AlphabetSoupCharity_Optimization2.h5")

C:\Users\daian\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
